In [58]:
from langchain.document_loaders.sitemap import SitemapLoader
import nest_asyncio
nest_asyncio.apply()

In [18]:
web_path = "https://alunodigital.estacio.br/sitemap.xml"
pattern = [r'^https://alunodigital\.estacio\.br/guiapratico.*']
loader = SitemapLoader(web_path=web_path, filter_urls=pattern)
docs = loader.load()

Fetching pages: 100%|##########| 30/30 [00:16<00:00,  1.80it/s]


In [67]:
docs[0].metadata

{'source': 'https://alunodigital.estacio.br/guiapratico/calouro',
 'loc': 'https://alunodigital.estacio.br/guiapratico/calouro',
 'lastmod': '2023-10-10'}

In [ ]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    separator = "\n\n",
    chunk_size = 512,
    chunk_overlap  = 0.25,
    length_function = len,
    is_separator_regex = False,
)

chunks = text_splitter.split_documents(docs);

In [64]:
import os
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient

customer_storage_connection_string = os.getenv("DOCUMENT_AZURE_STORAGE_CONNECTION_STRING")
container_name = os.getenv("DOCUMENT_AZURE_STORAGE_CONTAINER_NAME")

blob_service_client = BlobServiceClient.from_connection_string(customer_storage_connection_string)
container_client = blob_service_client.get_container_client(container=container_name)

if not container_client.exists():
    container_client.create_container()

# Upload strings from the list to blob storage as text files
for doc in docs:
    blob = doc.metadata['source'].split('.br/')[-1].replace('/', '_') + '.txt'
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob)
    
    # Upload the string as a text file
    blob_client.upload_blob(doc.page_content, overwrite=True)

In [38]:
import os
import openai
from dotenv import load_dotenv

load_dotenv(verbose=True)

openai.api_type = os.getenv("OPENAI_API_TYPE")
openai.api_version = os.getenv("OPENAI_API_VERSION")
openai.api_key = os.getenv("OPENAI_API_KEY")
openai.api_base = os.getenv("OPENAI_API_BASE")

text = docs[1].page_content

prompt = """
### Instructions
- You will receive a text. 
- You will rewrite the text. 
- Keep the text's language. 
- You will NOT ommit or summarize information. The content of the rewritten text must be faithful to the original

### Text to be rewritten
{}

### Rewritten text:""".format(text)

response = openai.Completion.create(
    engine="gpt-35-turbo-instruct",
    prompt=prompt,
    max_tokens=1000
)

In [41]:
for splitted_text in response.choices[0].text.split('\n'):
    print(splitted_text)



ALO, COMECE POR AQUI
Olá, nós o recebemos na Estácio! Estamos contentes em tê-lo conosco. Descubra tudo o que precisa saber sobre o início dos seus estudos em nosso ensino digital através dos passos iniciais que apresentaremos abaixo. A modalidade de ensino digital requer muita dedicação e disciplina, mas estaremos sempre ao seu lado neste percurso. Aqui, você é o protagonista dos seus estudos e desenvolvimento. Lembre-se de que estudar online não significa estudar sozinho. Participe, construa e colabore com o seu tutor e colegas!

Primeiros passos
Antes de tudo, há 5 temas essenciais que você precisa saber sobre a sua Graduação Digital! Vamos lá? 👇

Period PAR x Period ÍMPAR
Calendário Acadêmico
Campus Virtual - SIA
Sala de Aula Virtual - SAVA
APP Minha Estácio

Period PAR x Period ÍMPAR
Aqui na Estácio Digital, os alunos podem iniciar a graduação em diferentes épocas do ano. Dividimos esses períodos entre PAR e ÍMPAR. Para identificar em qual período você se encontra, acesse: Campu